<a href="https://colab.research.google.com/github/TajbeerAhamed/Assignment-1/blob/main/2125051007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*Conventional* Approach

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog

# Path to dataset
data_dir = '/content/drive/MyDrive/ML LAb PRoject/Dataset'

# Function to load dataset
def load_dataset(data_dir):
    labels = []
    images = []
    class_names = os.listdir(data_dir)
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, file)
            # Ensure only image files are processed
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                if img is not None:  # Ensure valid images
                    img = cv2.resize(img, (64, 64))
                    images.append(img)
                    labels.append(label)
    return np.array(images), np.array(labels), class_names

# Function for feature extraction using HOG
def extract_features(images):
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        hog_features = hog(gray,
                           orientations=9,
                           pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2),
                           block_norm='L2-Hys')
        features.append(hog_features)
    return np.array(features)



# Load the dataset
images, labels, class_names = load_dataset(data_dir)
print(f"Loaded {len(images)} images across {len(class_names)} classes: {class_names}")

# Extract HOG features
features = extract_features(images)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train SVM classifier
classifier = SVC(kernel='linear', C=1.0, random_state=42)
classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

Loaded 400 images across 4 classes: ['Truck', 'motorcycle', 'Bus', 'Car']
Accuracy: 0.6875
              precision    recall  f1-score   support

       Truck       0.67      0.54      0.60        26
  motorcycle       0.78      1.00      0.88        18
         Bus       0.54      0.78      0.64        18
         Car       0.90      0.50      0.64        18

    accuracy                           0.69        80
   macro avg       0.72      0.70      0.69        80
weighted avg       0.72      0.69      0.68        80



*CNN* Approach

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Load dataset
data_dir = '/content/drive/MyDrive/ML LAb PRoject/Dataset'

# Load dataset
def load_dataset(data_dir):
    labels = []
    images = []
    class_names = os.listdir(data_dir)
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        # Iterate through files within the class directory
        for file in os.listdir(class_dir):
            # Construct the full image path
            img_path = os.path.join(class_dir, file)
            # Check if it's a file and not a directory before loading
            if os.path.isfile(img_path) and file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
                img = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels), class_names


images, labels, class_names = load_dataset(data_dir)

# Normalize images
images = images / 255.0

# One-hot encode labels
labels = to_categorical(labels, num_classes=len(class_names))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 368ms/step - accuracy: 0.2139 - loss: 1.5048 - val_accuracy: 0.3594 - val_loss: 1.3603
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 182ms/step - accuracy: 0.5904 - loss: 1.3104 - val_accuracy: 0.3125 - val_loss: 1.3125
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 182ms/step - accuracy: 0.5327 - loss: 1.1284 - val_accuracy: 0.4219 - val_loss: 1.1303
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - accuracy: 0.6069 - loss: 0.9723 - val_accuracy: 0.6094 - val_loss: 1.0632
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.6527 - loss: 0.8534 - val_accuracy: 0.4531 - val_loss: 1.2828
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.7053 - loss: 0.7200 - val_accuracy: 0.6094 - val_loss: 0.8950
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 312ms/step - accuracy: 0.7984 - loss: 0.5748 - val_accuracy: 0.6875 - val_loss: 0.8616
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 312ms/step - accuracy: 0.9150 - loss: 0.4068 - val_accuracy: 0.5938 - val_loss: